# 信号処理の練習

## はじめに
### `soundfile`のインストール

## Guitar (string mode)

numpyを使う例として、音声の合成と解析を行う。

基底音: 正弦波(440 Hz, ド)

サンプリング周波数は20000 Hzとする。

* `time`は文字通り時間。20000個のarrayの中に時刻が書かれている。
* `phase`は位相(角度)。$2\pi$増えるとサイン波が0に戻る。440 Hzの場合は、1秒(20000サンプル)のあいだに$440\cdot 2\pi$だけ増加する。
* `wave`は波形。ここではコサイン波を作る。

In [ ]:
%matplotlib inline

import numpy as np
from math import pi
from IPython.display import Audio
import matplotlib.pyplot as plt

time = np.linspace(0, 1.0, 20000)
time

In [ ]:
plt.plot(time, time)

In [ ]:
phase = time* (440*pi*2)
wave = np.cos(phase)

#plot the wave
plt.xlim(0,0.02)
plt.ylim(-1,10)
plt.plot(time, phase)
plt.plot(time, wave)

Audio(wave, rate=20000)

減衰させる。減衰の時定数は0.5 (0.5秒で$1/e$まで減衰する)

In [ ]:
decay = np.exp(-time/0.5)

Audio(wave*decay, rate=20000)

2倍音。(振動数が2倍の音、1オクターブ上の音)

In [ ]:
wave = np.cos(2*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

3倍音。(振動数が3倍の音、1オクターブ上のソ)

In [ ]:
wave = np.cos(3*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

4倍音。(振動数が4倍の音、2オクターブ上の同音)

In [ ]:
wave = np.cos(4*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

基底音+倍音

In [ ]:
wave = np.cos(phase)
wave += 0.5**0.5*np.cos(2*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

基底音+倍音+3倍音

In [ ]:
wave = np.cos(phase)
wave += (1/2)**0.5*np.cos(2*phase)
wave += (1/3)**0.5*np.cos(3*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

基底音+倍音+3倍音+4倍音

In [ ]:
wave = np.cos(phase)
wave += (1/2)**0.5*np.cos(2*phase)
wave += (1/3)**0.5*np.cos(3*phase)
wave += (1/4)**0.5*np.cos(4*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

In [ ]:
wave = np.cos(phase)
wave += (1/2)**0.5*np.cos(2*phase)
wave += (1/3)**0.5*np.cos(3*phase)
wave += (1/4)**0.5*np.cos(4*phase)
wave += (1/3)**0.5*np.cos(5*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)

Audio(wave*decay, rate=20000)

上の波をフーリエ変換してパワースペクトル(周波数分布)を求める。

ここで利用するフーリエ変換の定義式は次のリンクを参照: https://docs.scipy.org/doc/numpy/reference/routines.fft.html#module-numpy.fft

フーリエ変換は、信号波$f(t)$に、いろんな正弦波(サイン波、コサイン波)を重ねて積分し、成分に分解する。
$$F(\omega)=\int_{-\infty}^\infty f(t)\cos(2\pi\omega t)\mathrm{d}t+i\int_{-\infty}^\infty f(t)\sin(2\pi\omega t)\mathrm{d}t=\int_{-\infty}^\infty f(t)\exp(2\pi i\omega t)\mathrm{d}t$$

$N$個の離散的なデータ列$f(k)$ $(k=[1..N])$の場合には、離散的フーリエ変換を用いる。
$$F(k)=\sum_{j=1}^N f(k)\cos\left({2\pi j k\over N}\right)+i\sum_{j=1}^N f(k)\sin\left({2\pi j k\over N}\right)=\sum_{j=1}^N f(k)\exp\left({2\pi j k\over N}\right)$$
(本によって微妙に定義が違う場合があるので注意)

1秒分(20000点)の実数データ列をフーリエ変換すると、結果も20000点得られる。フーリエ変換後に得られるarrayの`[1]`番目の値は、1 Hzの成分(実部がcos、虚部がsin)、`[2]`番目が、2 Hz、という具合に対応する。なお、もし元データが10秒分あるなら、`[1]`番目の成分は1/10=0.1 Hzに対応する。`[0]`番目は直流成分(振動しない成分、オフセット)を表す。

In [ ]:
spec = np.fft.fft(wave)

#plot the spectrum
plt.xlim(0,3000)
plt.plot(np.abs(spec)**2)

`abs()`関数は複素数の絶対値も計算できる。この場合、余弦波の振幅(実数)と正弦波の振幅(虚数)の二乗和を計算しているので、波の振幅の二乗=パワーをプロットしていることになる。分光器で測定するスペクトルも出力されているのは光の振幅の二乗=強度(パワー)である。

合成した波の成分に戻すことができた。ここでは音声データを分解したが、波データであれば光であれ音声であれ、こうしてスペクトルにに変換できる。

## Timpani (circular membrane modes)
膜の振動は、共鳴的ではない。固有振動が無理数比の成分を含み、濁った音になる。

基底音の周波数を120 Hzとする。

In [ ]:
time = np.linspace(0, 5, 20000*5)
phase = time * 120 * 2*pi
decay = np.exp(-time / 0.5)

wave = np.sin(phase)
# 倍音の強度はてきとう。
wave += (1/2)**0.5*np.sin(1.59*phase)
wave += (1/3)**0.5*np.sin(2.14*phase)
wave += (1/4)**0.5*np.sin(2.30*phase)
wave += (1/3)**0.5*np.sin(2.65*phase)
wave += (1/6)**0.5*np.sin(2.92*phase)
wave += (1/7)**0.5*np.sin(3.16*phase)
wave += (1/8)**0.5*np.sin(3.50*phase)
wave += (1/9)**0.5*np.sin(3.60*phase)

#plot the wave
plt.xlim(0,0.02)
plt.plot(time, wave*decay)


Audio(wave*decay, rate=20000)

In [ ]:
# waveは5秒のデータ
spec = np.fft.fft(wave)
spec = spec[:3000]

#plot the spectrum
# データが5秒分ある場合、波数の最小目盛は1/5波数となる。
wavenumbers = np.arange(0, 600, 1/5)
plt.plot(wavenumbers, np.abs(spec)**2)

## 宿題1

sampleaudio.wav (長さ1秒、サンプリング周波数20000 Hz)に含まれる音の基底振動数を調べよ。

wavファイルを読みこむには、`soundfile`モジュールを使うと便利。 (https://stackoverflow.com/questions/34416283/how-to-properly-decode-wav-with-python)

* Anacondaでsoundfileを使えるようにし、kernelを再起動する。

In [ ]:
import soundfile as sf
wave, samplerate = sf.read("sampleaudio.wav")

Audio(wave, rate=samplerate)

## もう少し長い音楽データを扱う

信号を加工する練習として、音楽データをいじってみる。

In [ ]:
import soundfile as sf
import numpy as np
from IPython.display import Audio

wave, samplerate = sf.read("MyWay.wav")
# samplerate = sound.frame_rate
# data = np.array(sound.get_array_of_samples())
print(np.min(wave))
left= wave[:,0]
right = wave[:,1]

samplerate

1秒間を44100分割して波形を記録していることがわかった。

左右を平均してモノラル再生。

In [ ]:
mono = (left+right)/2
Audio(mono, rate=samplerate)

左右トラックの相関を見てみよう。(データが多すぎるので、100個おき)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(left[::100],right[::100],'.')

当然だが、左右トラックはかなり相関が強い。では、左右を引き算すると?

In [ ]:
diff = left-right
Audio(diff, rate=samplerate)

音声の最初の部分をグラフ表示。

毎秒44100個もデータがあるので、100個に1個まで間引く。最初の3秒分。

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(mono[::100])

時刻を1/10秒(4410コマ)ずらして、少し弱くして音を重ねると、反響音(リバーブ)に聞こえる。

In [ ]:
reverb = mono.copy()
reverb[4410:] += mono[:-4410]*0.5
Audio(reverb, rate=samplerate)

ちょっと安っぽいので、ずらす長さを小さくし、何重にも重ねてみる。(反響音は指数関数的に減衰すると仮定)

In [ ]:
reverb = mono.copy() / 2 # 音をたくさん重ねるので、音量をあらかじめ小さくしておく。

delay=int(44100*0.05)
reverb[delay:] += mono[:-delay]/2
reverb[delay*2:] += mono[:-delay*2]/4
reverb[delay*3:] += mono[:-delay*3]/8
reverb[delay*4:] += mono[:-delay*4]/16
reverb[delay*5:] += mono[:-delay*5]/32
reverb[delay*6:] += mono[:-delay*6]/64
reverb[delay*7:] += mono[:-delay*7]/128
Audio(reverb, rate=samplerate)

上でやっているのはこんな感じの重ねあわせ。

In [ ]:
time=np.linspace(0,3,44100*3)
plt.plot(time, mono[:44100*3])
plt.plot(time+0.05, mono[:44100*3]/2)
plt.plot(time+0.10, mono[:44100*3]/4)
plt.plot(time+0.15, mono[:44100*3]/8)

これをたくさん行うことは、音声$f(t)$と、残響特性である指数関数$g=\exp(-at)$ (ただし$t>0$)の「畳み込み」を行うことにほかならない。

関数fとgの畳み込み(畳み込み積分)は次のように定義される。

$$(f*g)(t)=\int f(u)g(t-u)\mathrm{d}u$$

音声データ$f$がデルタ関数なら、畳み込み積分は$g$そのものになる。(例えば、風呂の中で手をたたけば、風呂の残響特性がわかる)そのため、$g$のことを音響学では「インパルス応答」と呼ぶ。

numpyの畳み込み積分をそのまま使い、音声に残響を付けてみよう。

In [ ]:
import numpy as np
time = np.linspace(0, 0.1, 4410)
response = np.exp(-time*30) # 0.1秒で1/e^3 = 1/20に減衰する指数関数
plt.plot(time,response)

In [ ]:
output = np.convolve(mono, response, mode='same')
Audio(output, rate=samplerate)

こもるばかりであまりうまくいかない。試しにインパルス応答だけ聞いてみるが、音らしく聞こえない。

In [ ]:
Audio(response, rate=samplerate)

調べてみると、インパルス応答を集めているサイト(www.openairlib.net/auralizationdb)があるらしい。そこで、教会のインパルス応答をもらってくる。

The Lady Chapel, St Albans Cathedral, England  https://en.wikipedia.org/wiki/St_Albans_Cathedral

In [ ]:
import soundfile as sf
response, samplerate = sf.read("stalbans_a_mono.wav")

Audio(response, rate=samplerate)

本物のインパルス応答は、確かに教会で手を叩いた時の残響に聞こえるから面白い(あたりまえだけど)。

In [ ]:
plt.plot(response)
plt.show()

#縦拡大
plt.ylim(-0.01,0.01)
plt.plot(response)
plt.show()

samplerate

samplerateも同じ44100なので、単純に音源にこれをたたみこんでやれば、教会で歌うMy Wayになるはず。

ただし、6秒もの残響の計算は時間がかかりすぎるので、はじめの1秒だけ使うことにする。
(PCが速い人は、全部使って試してみてもいい)

(フーリエ変換を使って畳み込み積分を爆速で計算する方法があるが、numpyはどうもそれを使っていないようだ。)

In [ ]:
song = np.convolve(mono, response[:44100], mode='same')

In [ ]:
Audio(song, rate=samplerate)

原曲と聴きくらべ。

In [ ]:
Audio(mono, rate=samplerate)

scipyにはFFT(高速フーリエ変換)を使った畳み込みがあった。 https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.fftconvolve.html#scipy.signal.fftconvolve


In [ ]:
import scipy.signal
fftsong = scipy.signal.fftconvolve(mono, response, mode='same')
Audio(fftsong, rate=samplerate)

畳み込み積分は、音響学だけでなく、様々な分野で非常に重要である。

* 光学におけるブレやボケの計算
* 電気回路における信号伝達
* 桁数の非常に大きな数同士のかけ算
* 信号へのノイズの重畳
* 量子力学における伝搬関数
* 液体論における積分方程式


## 宿題2

上で読みこんだmono (MyWay.mp3のモノラル音声)を使って、以下の処理を行うプログラムを書いて下さい。(できる範囲で)

1. 逆再生
2. 倍速再生 (データを1つおきに間引く。音が1オクターブ高くなる)
3. もとの音に0.2秒ずらした音を重ねて再生
4. 倍速再生 (音が高くならない)
5. 最大値、最小値、平均値を求める
6. 教会のインパルス応答を逆再生にして、monoに畳み込んで、変な残響をつくる

----

2番、誰かが話している時に、その人の声を0.2秒遅れで再生して聞かせると、その人は話が続けられなくなる、という研究があります。(https://srad.jp/story/12/03/04/1812249/ 2012年イグ・ノーベル賞https://scienceportal.jst.go.jp/news/newsflash_review/newsflash/2012/10/20121003_01.html)

----

4番、音が高くならないように高速再生する方法はいくつかありますが、一番簡単なのは、

* まず音楽を0.1秒ぐらいの長さのパケットに細分する。
* パケットを1つおきにえらんでつなぎなおす。

という方法です。例えば、元のデータが`[1,2,3,4,5,6,7,8,9,10, ... ,997,998,999,1000]`だとして、これを4データずつのパケットに分割し、`[1,2,3,4] [5,6,7,8] [9,10,11,12] ... [997 998 999 1000]`として、それらからひとつおきにえらんできてつないで`[1,2,3,4,9,10,11,12,17,18,19,20, ... ,993,994,995,996]`のようにするわけです。

実際のプログラム中では、元の音楽データ(mono, 20秒、882000データ)の半分の長さの0配列をあらかじめ準備し、そこにmonoの断片を書きこんでいきます。

In [ ]:
half = np.zeros(44100*10)
# 0.1秒分のデータ長
packetlen = 4410
packetnum = 100
# 最初の0.1秒をmonoからhalfに書き移す
half[0:packetlen] = mono[0:packetlen]
# 次の0.1秒
half[packetlen:packetlen*2] = mono[packetlen*2:packetlen*2+packetlen]
# これをpacketnum回くりかえす (for文を使う)

なお、ゆっくり再生したい場合は逆に、同じ断片を2個繰りかえして`[1,2,3,4,1,2,3,4,5,6,7,8,5,6,7,8,...]`のようにつなぎます。

2と4の技術を組みあわせると、再生速度がもとのままで、音だけ1オクターブ下げる、ということも可能です。(ボイスチェンジャー)